# 행정안전부 도로명 주소 API활용

In [1]:
import requests, json
import pandas as pd
from urllib.parse import quote

In [2]:
#API 키 가져오기
with open('도로명주소apikey.txt') as file:
    road_key = file.read()

In [3]:
# url 
quote('신촌역')

'%EC%8B%A0%EC%B4%8C%EC%97%AD'

In [4]:
base_url = 'https://www.juso.go.kr/addrlink/addrLinkApi.do'
params1 = f'confmKey={road_key}&currentPage=1&countPerPage=10'
params2 = f'keyword={quote("신촌역")}&resultType=json'
url = f'{base_url}?{params1}&{params2}'

# open API를 호출해서 결과 받아오기

In [5]:
result = requests.get(url)
result.status_code

200

In [6]:
result.text

'{"results":{"common":{"errorMessage":"정상","countPerPage":"10","totalCount":"38","errorCode":"0","currentPage":"1"},"juso":[{"detBdNmList":"","engAddr":"4 Sinchonnyeok-ro, Seodaemun-gu, Seoul","rn":"신촌역로","emdNm":"대현동","zipNo":"03766","roadAddrPart2":" (대현동)","emdNo":"01","sggNm":"서대문구","jibunAddr":"서울특별시 서대문구 대현동 101-22","siNm":"서울특별시","roadAddrPart1":"서울특별시 서대문구 신촌역로 4","bdNm":"","admCd":"1141011200","udrtYn":"0","lnbrMnnm":"101","roadAddr":"서울특별시 서대문구 신촌역로 4 (대현동)","lnbrSlno":"22","buldMnnm":"4","bdKdcd":"0","liNm":"","rnMgtSn":"114103112007","mtYn":"0","bdMgtSn":"1141011200101010022024077","buldSlno":"0"},{"detBdNmList":"","engAddr":"7 Sinchonnyeok-ro, Seodaemun-gu, Seoul","rn":"신촌역로","emdNm":"대현동","zipNo":"03780","roadAddrPart2":" (대현동)","emdNo":"01","sggNm":"서대문구","jibunAddr":"서울특별시 서대문구 대현동 104-5 신촌푸르지오시티","siNm":"서울특별시","roadAddrPart1":"서울특별시 서대문구 신촌역로 7","bdNm":"신촌푸르지오시티","admCd":"1141011200","udrtYn":"0","lnbrMnnm":"104","roadAddr":"서울특별시 서대문구 신촌역로 7 (대현동)","lnbrSlno":"5","bu

In [7]:
# json data를 python에서 읽을수 있도록 변환
res = json.loads(result.text)

In [8]:
res.keys()

dict_keys(['results'])

In [9]:
res['results'].keys()

dict_keys(['common', 'juso'])

In [10]:
type(res['results']['juso'])

list

In [11]:
len(res['results']['juso'])

10

In [12]:
res['results']['juso'][0]

{'detBdNmList': '',
 'engAddr': '4 Sinchonnyeok-ro, Seodaemun-gu, Seoul',
 'rn': '신촌역로',
 'emdNm': '대현동',
 'zipNo': '03766',
 'roadAddrPart2': ' (대현동)',
 'emdNo': '01',
 'sggNm': '서대문구',
 'jibunAddr': '서울특별시 서대문구 대현동 101-22',
 'siNm': '서울특별시',
 'roadAddrPart1': '서울특별시 서대문구 신촌역로 4',
 'bdNm': '',
 'admCd': '1141011200',
 'udrtYn': '0',
 'lnbrMnnm': '101',
 'roadAddr': '서울특별시 서대문구 신촌역로 4 (대현동)',
 'lnbrSlno': '22',
 'buldMnnm': '4',
 'bdKdcd': '0',
 'liNm': '',
 'rnMgtSn': '114103112007',
 'mtYn': '0',
 'bdMgtSn': '1141011200101010022024077',
 'buldSlno': '0'}

In [13]:
res['results']['juso'][0]['roadAddr']

'서울특별시 서대문구 신촌역로 4 (대현동)'

# 여러검색어 보내기

In [14]:
# 신촌역, 홍대역, 여의도역, 서대문구청
stations = '신촌역, 홍대입구역, 여의도역, 서대문구청'.split(', ')
stations

['신촌역', '홍대입구역', '여의도역', '서대문구청']

In [15]:
# 반복문
base_url = 'https://www.juso.go.kr/addrlink/addrLinkApi.do'
params1 = f'confmKey={road_key}&currentPage=1&countPerPage=10'
# params2 = f'keyword={quote("신촌역")}&resultType=json'

url = f'{base_url}?{params1}&{params2}'
road_addr_list = []
for station in stations:
    params2 = f'keyword={quote(station)}&resultType=json'
    url = f'{base_url}?{params1}&{params2}'
    result = requests.get(url)
    if result.status_code == 200:
        res = json.loads(result.text)
        road_addr_list.append(res['results']['juso'][0]['roadAddr'])
    else:
        print(result.status_code)

In [16]:
road_addr_list

['서울특별시 서대문구 신촌역로 4 (대현동)',
 '서울특별시 마포구 양화로 지하160 (동교동)',
 '서울특별시 영등포구 여의나루로 지하40 (여의도동)',
 '서울특별시 서대문구 연희로 247 (연희동)']

In [17]:
df = pd.DataFrame({'이름' : stations, '주소':road_addr_list})
df.to_csv('test_data/신촌주변지하철역.csv', index=False)

In [18]:
pd.read_csv('test_data/신촌주변지하철역.csv')

,이름,주소
0,신촌역,서울특별시 서대문구 신촌역로 4 (대현동)
1,홍대입구역,서울특별시 마포구 양화로 지하160 (동교동)
2,여의도역,서울특별시 영등포구 여의나루로 지하40 (여의도동)
3,서대문구청,서울특별시 서대문구 연희로 247 (연희동)
